In [1]:
# import tensorflow.contrib.eager as tfe



import librosa
import librosa.display
import os
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random
import keras
# import datetime as dt
from datetime import datetime
import time
# import datetime.datetime




from keras import layers
from keras import models
# from keras import optimizers
from tensorflow.keras import optimizers

from keras.applications.densenet import DenseNet121, DenseNet169
#from keras.applications.nasnet import NASNetLarge
# from keras_efficientnets import EfficientNetB7
from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints
from keras.layers.normalization import BatchNormalization 
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from keras import backend as K






tf.set_random_seed(1234)
random.seed(100)

os.environ["CUDA_VISIBLE_DEVICES"]="1"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)


/usr/local/lib/python3.6/dist-packages/numba/errors.py:137: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)typ

In [2]:

def score_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

def dot_product(x, kernel):
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    
def get_unique_classes(input_directory,files):

    unique_classes=set()
    for f in files:
        g = f.replace('.mat','.hea')
        input_file = os.path.join(input_directory,g)
        with open(input_file,'r') as f:
            for lines in f:
                if lines.startswith('#Dx'):
                    tmp = lines.split(': ')[1].split(',')
                    for c in tmp:
                        unique_classes.add(c.strip())
    return sorted(unique_classes)

def one_hot_encoding(one_hot_vector,y, class2index):
    ind=class2index[y]
    one_hot_vector[ind]=1
    return one_hot_vector

# Search for multi-label subjects
def searching_overlap(input_directory,class2index, input_file_names):
    multiclasses=[]
    multisubjects=[]
    number = []
    for file in input_file_names:
        f=file
        g = f.replace('.mat','.hea')
        input_file = os.path.join(input_directory,g)
        with open(input_file,'r') as f:
            for lines in f:
                if lines.startswith('#Dx'):
                    tmp = lines.split(': ')[1].split(',')
                    if len(tmp)>1:
                        one_hot_vector = [0]*(len(class2index))
                        for c in tmp:
                            one_hot_vector = one_hot_encoding(one_hot_vector, c.strip(), class2index)
                        multiclasses.append(one_hot_vector)
                        multisubjects.append(g)
                        number.append(len(tmp))
    return multisubjects, multiclasses, number

def block_feature(sequence_en, minimum_len): 
    new_en = []
    if len(sequence_en) > minimum_len:  # 길이가 minimum보다 긴 경우
        start = random.randint(0,len(sequence_en)-minimum_len)
        #print(start)
        new_en = sequence_en[start:start+minimum_len]
    elif len(sequence_en) == minimum_len: # 길이가 minimum
        new_en = sequence_en
    else: 
        assert len(sequence_en) <= minimum_len
    return new_en

def exploratory_look(input_directory,file, class2index):
    classes = []
    f = file
    g = f.replace('.mat','.hea')
    input_file = os.path.join(input_directory,g)
    with open(input_file,'r') as f:
        for lines in f:
            if lines.startswith('#Dx'):
                tmp = lines.split(': ')[1].split(',')
                print(tmp, len(tmp))
    return tmp     

# Get classes of sorted file names
def get_labels(input_directory,file, class2index):
    f = file
    g = f.replace('.mat','.hea')
    input_file = os.path.join(input_directory,g)
    with open(input_file,'r') as f:
        for lines in f:
            if lines.startswith('#Dx'):
                tmp = lines.split(': ')[1].split(',')
                one_hot_vector = [0]*(len(class2index))
                for c in tmp:
                    one_hot_vector = one_hot_encoding(one_hot_vector, c.strip(), class2index)
                
    return one_hot_vector

def randextract_mels(curr_step, batch_size, data, mel_directory, class2index, minimum_len, x_mean_final, x_std_final):
    mel_files = []
    classes = []
    start = batch_size*curr_step
    end = batch_size*(curr_step+1)
    curr_file_indices = data[start:end]
    for file in curr_file_indices:
        tmp_file = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
        clip_file = block_feature(tmp_file, minimum_len)
        #print(clip_file.shape)
        #clip_file = tmp_file[:minimum_len]
        clip_file -= x_mean_final
        clip_file /= x_std_final
        mel_files.append(clip_file)
        label = get_labels(input_directory, file, class2index)
        classes.append(label)
    
    # YJS added for ABN -> should calculate 2 losses
#     classes_abn = [classes,classes]
    
    concat = list(zip(mel_files, classes))
#     concat = list(zip(mel_files, classes_abn))
    random.shuffle(concat)
    mel_files, classes = zip(*concat)
    return mel_files, classes

def train(data_train, mel_directory, batch_size, class2index, minimum_len, x_mean_final, x_std_final): 
    loss=[]
#     acc = []
    f1 = []

    total_steps = int(np.ceil(len(data_train)/batch_size))
    for curr_step in range(total_steps):
        batch_mels, batch_labels = randextract_mels(curr_step, batch_size, data_train, mel_directory, class2index, minimum_len, x_mean_final, x_std_final)
        batch_mels = np.asarray(batch_mels)
        batch_labels2 = np.asarray(np.squeeze(batch_labels))
        batch_labels = [batch_labels2, batch_labels2]
#         print(batch_labels.shape)
#         print(batch_labels)

# return of train
# 0 = total loss (attention branch + perception branch)
# 1 = loss of attention pred
# 2 = loss of perception pred
# 3 = f1 of attention pred
# 4 = f1 of perception pred     

        train_tmp = model.train_on_batch(batch_mels, batch_labels)
#         print(train_tmp)
        loss_ = train_tmp[0]/2
        f1_ = np.mean((train_tmp[3], train_tmp[4]))
        loss.append(loss_)
#         acc.append(train_tmp[1])
        f1.append(f1_)

    loss = np.mean(np.array(loss))
#     acc = np.mean(np.array(acc))
    f1 = np.mean(np.array(f1))
    return loss, f1

def train_cam(data_train, mel_directory, batch_size, class2index, minimum_len, x_mean_final, x_std_final): 
    loss=[]
#     acc = []
    f1 = []

    total_steps = int(np.ceil(len(data_train)/batch_size))
    for curr_step in range(total_steps):
        batch_mels, batch_labels = randextract_mels(curr_step, batch_size, data_train, mel_directory, class2index, minimum_len, x_mean_final, x_std_final)
        batch_mels = np.asarray(batch_mels)
        
        batch_labels = np.asarray(np.squeeze(batch_labels))
#         batch_labels2 = np.asarray(np.squeeze(batch_labels))
#         batch_labels = [batch_labels2, batch_labels2]
# #         print(batch_labels.shape)
#         print(batch_labels)

# return of train
# 0 = total loss (attention branch + perception branch)
# 1 = loss of attention pred
# 2 = loss of perception pred
# 3 = f1 of attention pred
# 4 = f1 of perception pred     

        train_tmp = model.train_on_batch(batch_mels, batch_labels)
#         print(train_tmp)
        loss_ = train_tmp[0]
        f1_ = train_tmp[1]
        loss.append(loss_)
#         acc.append(train_tmp[1])
        f1.append(f1_)

    loss = np.mean(np.array(loss))
#     acc = np.mean(np.array(acc))
    f1 = np.mean(np.array(f1))
    return loss, f1

def test_cam(data, mel_directory, input_directory, class2index, minimum_len, model, x_mean_final, x_std_final):
    scores = []
    predicted_labels=[]
    accuracy=np.zeros(len(data))
    #total_loss=[]
    total_acc = 0
    total_f1 = 0
    
    mel_files = []
    classes = []
    for i, file in enumerate(data):
        tmp_file = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
        steps = int(np.floor(tmp_file.shape[0]/minimum_len))
        mel_files = []
        for block in range(steps): # 128개씩 쪼갠 블럭 단위로 predict
            start = block*minimum_len
            end = (block+1)*minimum_len
            clip_file = tmp_file[start:end]
            clip_file -= x_mean_final
            clip_file /= x_std_final
            mel_files.append(clip_file)
        mel_files = np.asarray(mel_files)
        logit = model.predict(mel_files)
#         print(len(logit))
        logit = np.mean(logit, axis=0)
#         logit = np.mean(logit, axis=0)
#         print(logit)
        pred = np.argmax(logit)
#         print('Pred={}'.format(pred))
        
        label = np.argmax(get_labels(input_directory, file, class2index))
#         print('Label={}'.format(label))
        #f1 = f1_score(label, logit)
        #print(pred, label)
        if pred == label:
            acc = 1
        else:
            acc = 0
        total_acc += acc
        #total_f1 += f1
    final_acc = total_acc / i
    #final_f1 = total_f1 / i
    return final_acc#, final_f1

batch_size = 32#16#20#32#5#2#1#10#32

minimum_len = 2880
epochs = 1000
loss_function = 'categorical_crossentropy'
activation_function = 'softmax'
rootdir = '../'
date = datetime.today().strftime("%Y%m%d")
input_directory = os.path.join(rootdir, 'Training_WFDB')
mel_name = 'Raw_data_20200424' 
mel_directory = os.path.join(rootdir, mel_name)
results_directory = os.path.join(rootdir, 'results_'+date+'_0_CAM_map_model')
if not os.path.isdir(input_directory):
    os.mkdir(input_directory)
if not os.path.isdir(mel_directory):
    os.mkdir(mel_directory)
if not os.path.isdir(results_directory):
    os.mkdir(results_directory)
        
input_files = []
for f in os.listdir(input_directory):
    if os.path.isfile(os.path.join(input_directory, f)) and not f.lower().startswith('.') and f.lower().endswith('mat'):
        input_files.append(f)
input_file_names = sorted(input_files)

unique_classes = get_unique_classes(input_directory, input_files)
class2index = {}
for a, b in enumerate(unique_classes):
    class2index[b] = a
    
classes_orig= [x.replace('.mat', '.hea') for x in input_file_names] # total subjects
classes_multi, _, _ = searching_overlap(input_directory,class2index, input_file_names)
classes_single = [x for x in classes_orig if x not in classes_multi]
classes_single = [x.replace('.hea', '.mat') for x in classes_single]

# double-checking if classes_single have single-label
a, b, c  = searching_overlap(input_directory,class2index,classes_single)

# we can safely use classes_single as input_file_names
input_file_names = classes_single
random.shuffle(input_file_names)
np.shape(input_file_names)

x_mean_all = []
x_std_all = []
for file in input_file_names:
    x = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
    x_mean = [np.mean(x[:,0]), np.mean(x[:,1]), np.mean(x[:,2]), np.mean(x[:,3]), np.mean(x[:,4]), np.mean(x[:,5]),
             np.mean(x[:,6]), np.mean(x[:,7]), np.mean(x[:,8]), np.mean(x[:,9]), np.mean(x[:,10]), np.mean(x[:,11])]
    
    x_std = [np.std(x[:,0]), np.std(x[:,1]), np.std(x[:,2]), np.std(x[:,3]), np.std(x[:,4]), np.std(x[:,5]),
             np.std(x[:,6]), np.std(x[:,7]), np.std(x[:,8]), np.std(x[:,9]), np.std(x[:,10]), np.std(x[:,11])]
    #print(x_mean)
    x_mean_all.append(x_mean)
    x_std_all.append(x_std) # yjs corrected on 2020-05-25
x_mean_final = np.mean(x_mean_all, axis=0)
x_std_final = np.mean(x_std_all, axis=0)
print(x_mean_final)

data, data_test = train_test_split(input_file_names, test_size = 0.2, train_size = 0.8, shuffle=True)
data_train, data_val = train_test_split(data, test_size = 0.25, train_size = 0.75, shuffle=True)
print(np.shape(data_train), np.shape(data_val), np.shape(data_test))


main_input = Input(shape=(minimum_len,12), dtype='float32', name='main_input')

branch_pred = []

[-2.24290386e-03 -4.58280585e-05  4.31697309e-03 -3.00174693e-03
 -2.36609229e-04  1.28997408e-03  2.17347589e-04 -7.99152384e-04
 -3.42993744e-03 -1.69711686e-03  1.27138164e-03  1.94670545e-03]
(3840,) (1280,) (1281,)



In [3]:
from ABNmodules import *

# def get_custom_model(input_shape, n_classes, minimum_len, target_classes, out_ch=256, n=18):
# model = get_custom_model((None, 12), 9, minimum_len, 1, out_ch=256, n=18)
# model = get_model((None, 12), 9, n=7)

model = cam_model((None, 12), 9, minimum_len, out_ch=256, n=18)
model.summary()
model.compile(loss=loss_function,
              optimizer=optimizers.Adam(lr=1e-5),           
              metrics=[score_f1])



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_image (InputLayer)     [(None, None, 12)]        0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 64)          2368      
_________________________________________________________________
batch_normalization (BatchNo (None, None, 64)          256       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 64)          12352     
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 64)          256       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 64)          0         
_________

In [4]:
# DISP DATETIME FOR CHECKING TIME
start = time.time()
val_acc_sum=[]
train_loss_sum=[]
train_acc_sum=[]
val_loss_sum=[]
val_acc_min = 0
print(results_directory)

for num_epoch in range(epochs):
    random.shuffle(data_train)
    train_loss, train_f1 = train_cam(data_train, mel_directory, batch_size, class2index, minimum_len, x_mean_final, x_std_final)
    print('\nEpoch',num_epoch+1,'train_loss:',f'{train_loss:.3f}','train_f1:',f'{train_f1:.3f}',"\t")
    model_output = "ECG_ABN_E%02dL%.2f" % (num_epoch, train_loss)
    save_name = os.path.join(results_directory, model_output)
    
    val_acc = test_cam(data_val, mel_directory, input_directory, class2index, minimum_len, model, x_mean_final, x_std_final)

    if val_acc > val_acc_min:
        val_acc_min = val_acc
        model.save(save_name)
    print('\nValidation', num_epoch+1, 'valid_acc:',f'{val_acc:.3f}', 'best_acc:',f'{val_acc_min:.3f}', "\t")
print(time.time() - start)

../results_20200528_0_CAM_map_model

Epoch 1 train_loss: 1.889 train_f1: 0.181 	
[0.26832724 0.04909244 0.07273417 0.08352742 0.14125267 0.08389777
 0.07556184 0.10066738 0.12493907]
[0.08595996 0.09382538 0.10405986 0.12578425 0.15558635 0.09310105
 0.12421987 0.12303981 0.09442355]
[0.06928132 0.05212577 0.11389734 0.11231586 0.07169555 0.35327452
 0.04022483 0.09257642 0.09460835]
[0.05275635 0.06083632 0.13124822 0.16406363 0.1200706  0.18261099
 0.09926398 0.08341287 0.10573702]
[0.0818795  0.09841281 0.10822488 0.17774169 0.14067721 0.08290008
 0.08005281 0.1261734  0.10393761]
[0.07895135 0.08123837 0.11898649 0.20389554 0.13209432 0.08817057
 0.0747747  0.12286277 0.09902591]
[0.09258166 0.10469186 0.13291365 0.1431793  0.11809072 0.1043674
 0.06383619 0.1240289  0.11631034]
[0.08954869 0.10324807 0.10817719 0.13599667 0.14425667 0.08437327
 0.09903576 0.1277001  0.10766356]
[0.08505492 0.07649001 0.11635425 0.18930542 0.13779262 0.10495837
 0.06246693 0.10960127 0.1179762 ]
[0

[0.0910183  0.08056527 0.11228293 0.17835943 0.13084601 0.09758498
 0.06748022 0.12985232 0.11201055]
[0.08105391 0.06608701 0.10609457 0.16144887 0.11767101 0.11104442
 0.08500185 0.18096304 0.09063532]
[0.15855138 0.10582726 0.11052669 0.13052851 0.13011593 0.08259845
 0.06112625 0.11070152 0.11002403]
[0.09127862 0.07671408 0.10957025 0.16597427 0.12829909 0.10645965
 0.08617111 0.13653618 0.09899677]
[0.06243084 0.03336703 0.30208534 0.07923768 0.13890406 0.1447908
 0.02500287 0.11824121 0.0959402 ]
[0.1030428  0.1012584  0.10773543 0.14255211 0.15870325 0.07807874
 0.06715004 0.13162494 0.10985431]
[0.08300517 0.07338615 0.11320221 0.14656843 0.13468139 0.09076832
 0.1566764  0.1003986  0.10131329]
[0.0793554  0.08483945 0.11884104 0.17710106 0.1272179  0.11205956
 0.08676141 0.1107299  0.10309437]
[0.08807223 0.10210425 0.10380894 0.15477371 0.15201974 0.07324609
 0.09307615 0.12825216 0.10464679]
[0.10799614 0.0988633  0.10229501 0.13038036 0.13218607 0.09301442
 0.06761461 0.14

[0.09153531 0.09608745 0.10279992 0.18788897 0.13698673 0.08084423
 0.07169333 0.12461191 0.10755216]
[0.0913683  0.07852151 0.09239962 0.15304403 0.11886904 0.11542477
 0.05058427 0.1846015  0.11518693]
[0.05702421 0.05411014 0.1477846  0.11967287 0.1519013  0.07851415
 0.21390271 0.09613047 0.08095962]
[0.19820604 0.08476051 0.08626731 0.09965259 0.12047166 0.10317928
 0.06279842 0.1328162  0.11184806]
[0.06489262 0.04975502 0.12404263 0.12118334 0.14814083 0.0853799
 0.21635652 0.09674107 0.09350807]
[0.05839283 0.07219529 0.12208601 0.13613434 0.13485357 0.07998564
 0.16820222 0.12312391 0.10502609]
[0.07941343 0.07015456 0.10595728 0.20781578 0.14443843 0.09027804
 0.07953496 0.13654442 0.0858631 ]
[0.12386855 0.07243992 0.10888541 0.1256174  0.12576157 0.09733766
 0.06542256 0.16251698 0.11814994]
[0.08890284 0.0911391  0.11082376 0.14373384 0.12170223 0.1144329
 0.07493141 0.13843429 0.1158996 ]
[0.08178162 0.07506015 0.12016823 0.1141382  0.13945727 0.08744608
 0.15869512 0.113

[0.07994232 0.08894223 0.10854913 0.19291899 0.13829288 0.09104441
 0.07535788 0.12308525 0.10186695]
[0.08295362 0.06616677 0.104105   0.15082897 0.10607009 0.1545187
 0.07202762 0.16841793 0.0949113 ]
[0.06682613 0.07480112 0.10813545 0.1966083  0.12077152 0.11368771
 0.08875737 0.12098413 0.10942824]
[0.09631385 0.104766   0.10393615 0.1576862  0.14714934 0.08452632
 0.08765207 0.12427498 0.09369506]
[0.07435597 0.10076339 0.10204022 0.19123054 0.13957897 0.08610689
 0.07817423 0.1307589  0.09699091]
[0.09273755 0.10915123 0.10326475 0.17045127 0.14974944 0.08213157
 0.06471491 0.12297715 0.10482214]
[0.06798453 0.0585576  0.24684115 0.08906408 0.13115111 0.12833108
 0.04804058 0.11802787 0.11200202]
[0.08486517 0.08195969 0.11584088 0.15218103 0.13716082 0.10413175
 0.08777645 0.13398278 0.1021014 ]
[0.07276387 0.08606693 0.09114531 0.19721505 0.14109148 0.09470323
 0.09246419 0.12764543 0.09690449]
[0.1127931  0.07611081 0.10628363 0.14563388 0.15698785 0.08868906
 0.07187356 0.14

[0.06647512 0.06084069 0.10850316 0.18910141 0.1345774  0.07711443
 0.12866914 0.11958456 0.11513414]
[0.06504834 0.0721148  0.10493093 0.21897316 0.12089819 0.08806485
 0.09990191 0.11322882 0.11683905]
[0.08921262 0.04942175 0.10663749 0.13426302 0.14563304 0.12132435
 0.09408398 0.14333123 0.11609253]
[0.08566331 0.09795415 0.1152275  0.14751303 0.14889315 0.08760346
 0.09223203 0.12523872 0.09967466]
[0.09378141 0.11026865 0.11742482 0.18368626 0.13419123 0.07807348
 0.08644986 0.10120729 0.09491695]
[0.06327547 0.07496758 0.2565568  0.10473601 0.12910187 0.12382787
 0.06452323 0.08142384 0.10158732]
[0.13620378 0.07767995 0.09961492 0.11074144 0.13945925 0.07502218
 0.09452027 0.12386563 0.14289257]
[0.07888401 0.05867003 0.1073252  0.18513265 0.1195457  0.1009319
 0.07716004 0.16635716 0.1059933 ]
[0.09563083 0.07653282 0.1116363  0.14553265 0.13752007 0.09887113
 0.10618605 0.09602907 0.13206106]
[0.10275329 0.10078085 0.1087329  0.17034759 0.12035961 0.08037716
 0.09317787 0.11

[0.07009829 0.07306917 0.11132397 0.17871076 0.13752891 0.0797948
 0.10404936 0.12628774 0.11913699]
[0.12191708 0.08898231 0.08258967 0.14820409 0.12081733 0.09896531
 0.04732984 0.15957282 0.13162152]
[0.0845551  0.09852324 0.11485739 0.14181432 0.14637902 0.08756661
 0.09895287 0.12673132 0.10062013]
[0.1189215  0.08663479 0.12237824 0.0891773  0.17081389 0.10198443
 0.08938813 0.14498858 0.07571315]
[0.13246532 0.06445374 0.04962368 0.11453218 0.11669653 0.18483853
 0.10856996 0.09239589 0.13642411]
[0.07126002 0.07912715 0.12248653 0.1440275  0.13803998 0.09540494
 0.10415359 0.12151972 0.1239806 ]
[0.10011706 0.11113489 0.11226621 0.14613701 0.14082964 0.08816233
 0.07677185 0.12457544 0.10000554]
[0.07642423 0.09105717 0.10957763 0.17556378 0.14895976 0.08888729
 0.08741201 0.13229784 0.08982029]
[0.08377934 0.11152698 0.09508219 0.18612957 0.14764695 0.07271214
 0.06731427 0.1318414  0.10396712]
[0.06814278 0.05184584 0.13819753 0.09578303 0.17655706 0.07987633
 0.21143806 0.08

[0.07287696 0.07030039 0.10440102 0.23335418 0.11396696 0.09250785
 0.07011317 0.11804415 0.12443537]
[0.08484489 0.06372859 0.11744194 0.11038104 0.15742177 0.09599501
 0.14356756 0.12351568 0.1031035 ]
[0.09015019 0.06938951 0.1006197  0.15648869 0.15358265 0.09954973
 0.06949583 0.15781945 0.10290425]
[0.0945776  0.07174586 0.09382659 0.15323675 0.129113   0.10935438
 0.06479094 0.16876185 0.11459304]
[0.07241823 0.09123084 0.09598929 0.14752899 0.14485186 0.09400009
 0.13704196 0.12469409 0.0922446 ]
[0.06564055 0.04240156 0.11448461 0.12074377 0.17242865 0.08150181
 0.18892552 0.10348732 0.11038622]
[0.07740413 0.09047338 0.10206117 0.19704796 0.14618248 0.0844191
 0.07872179 0.12465039 0.09903958]
[0.0866095  0.09905089 0.08718501 0.18536636 0.14495897 0.09431174
 0.0588128  0.13462943 0.10907531]
[0.08060142 0.10131812 0.09632559 0.18431823 0.15282714 0.07879689
 0.08217633 0.12715673 0.09647954]
[0.07749213 0.07090309 0.11662079 0.13449591 0.14480518 0.09484018
 0.13377331 0.11

[0.08479913 0.06111486 0.10450753 0.16204973 0.1109859  0.10061196
 0.06990734 0.18542725 0.12059638]
[0.05846075 0.0517805  0.1314595  0.10357609 0.15572856 0.08096254
 0.21795417 0.12054721 0.07953072]
[0.08829366 0.07384952 0.09199361 0.19666861 0.14528657 0.10445233
 0.06309999 0.1377368  0.09861892]
[0.08303004 0.06620514 0.10373896 0.19704147 0.14108188 0.09089427
 0.07633637 0.14243314 0.09923875]
[0.10113462 0.08915677 0.10096887 0.14327005 0.13540587 0.10446385
 0.05821732 0.15776765 0.10961508]
[0.0646725  0.09223312 0.10929608 0.14100228 0.14583668 0.08240717
 0.13879696 0.12847672 0.09727845]
[0.21936607 0.08785731 0.07767176 0.07488421 0.13160783 0.100321
 0.06111692 0.13049902 0.11667588]
[0.07751945 0.05957906 0.11427468 0.15724874 0.11587758 0.11078853
 0.07520524 0.18060276 0.10890391]
[0.07782638 0.09784472 0.09880229 0.20313855 0.14515637 0.08220879
 0.07228032 0.12054646 0.10219616]
[0.09022286 0.09744022 0.12210821 0.2122805  0.12561266 0.10590004
 0.07044695 0.092

KeyboardInterrupt: 